In [47]:
import random
import networkx as nx
import math
import numpy as np
import pyvista as pv
from pyvista import examples

In [48]:
earth = examples.download_topo_global().triangulate().decimate(0.98)
land = examples.download_topo_land().triangulate().decimate(0.98)

In [49]:
class Transaction:
    """ A transaction on the Ethereum network """
    def __init__(self, sender,fee, timestamp = 0,dataset = None):
        self.sender = sender
        self.fee = fee
        self.metrics = {}
        self.send_timestamp = timestamp
        if dataset!= None:
            ids = np.random.randint(low=0, high=dataset.n_points-1,size=n_nodes)
            self.position = earth.find_closest_point(dataset.points[ids])

In [50]:
def generate_points(n_nodes):
    """A helper to make a 3D NumPy array of points (n_points by 3)"""
    dataset = land
    ids = np.random.randint(low=0, high=dataset.n_points-1,size=n_nodes)
    return dataset.points[ids]

def lenght_of_path(distance,node_path):
    dist  = 0
    for i in range(0,len(node_path)-1):
        dist += distance[node_path[i]][node_path[i+1]]
    return dist

In [51]:
nodes_sphere = []
position = generate_points(12)
for node in position:
    nodes_sphere.append(earth.find_closest_point(node))

In [53]:
def network_nodes(n_peers,nodes_sphere):
    peers = {}
    distance = {}
    for node in nodes_sphere:
        distance[node] = {}
    for node in nodes_sphere:
        set_nodes = [element for element in nodes_sphere if element!=node]
        distance_node = {peer:earth.geodesic_distance(node,peer) for peer in set_nodes}
        ordered = sorted(set_nodes, key=lambda peer:distance_node[peer])
        peers[node] = ordered[:n_peers]
        for peer in peers[node]:
            distance[node][peer] = distance_node[peer]
            distance[peer][node] = distance_node[peer]
    G = nx.Graph()
    for node in nodes_sphere:
        for node2 in peers[node]:
            G.add_edge(node,node2,weight = distance[node][node2])
    for node in G.nodes():
        for node2 in G.nodes:
            distance[node][node2] = lenght_of_path(distance,nx.shortest_path(G,node,node2))
    return peers,distance, G

In [54]:
peers, distance, graph = network_nodes(n_peers, nodes_sphere)

In [ ]:
p = pv.Plotter(notebook=0)
p.add_point_labels(position, labels = range(0,12),font_size=20)
p.add_mesh(position, line_width=1000, color="black", label="Geodesic Path")
for node in peers.keys():
    for peer in peers:
        p.add_mesh(earth.geodesic(node,peer),line_width=4,color='blue')
p.add_mesh(land,show_edges=False)
#p.add_mesh(earth)
p.add_legend()
p.camera_position = [(3.5839785524183934, 2.3915238111304924, 1.3993738227478327),
 (-0.06842917033182638, 0.15467201157962263, -0.07331693636555875),
 (-0.34851770951584765, -0.04724188391065845, 0.9361108965066047)]

p.show()